In [1]:
pip install rdflib

Note: you may need to restart the kernel to use updated packages.


In [35]:
from rdflib import Graph
import pandas as pd
import numpy as np

# TTL 파일 목록
default_ttl_files = [
    "./data/administrative-area.ttl",
    "./data/bus-station.ttl", 
    "./data/electronic-car.ttl",
    "./data/Income-average.ttl",
    "./data/welfare.ttl"
]

def load_graph(ttl_files):
    """
    주어진 TTL 파일들을 하나의 Graph에 로드
    """
    g = Graph()
    for file in ttl_files:
        try:
            g.parse(file, format="ttl")
            print(f"Loaded: {file}")
        except Exception as e:
            print(f"Error loading {file}: {e}")
    return g

def run_sparql_query(graph, query):
    """
    그래프와 쿼리를 받아서 실행 후 DataFrame으로 반환
    """
    results = graph.query(query)
    cols = results.vars  # SELECT 절 변수명
    data = []

    for row in results:
        data.append([str(row.get(var)) for var in cols])

    df = pd.DataFrame(data, columns=[str(var) for var in cols])
    return df

In [10]:
g = load_graph(default_ttl_files)

Loaded: ./data/administrative-area.ttl
Loaded: ./data/bus-station.ttl
Loaded: ./data/electronic-car.ttl
Loaded: ./data/Income-average.ttl
Loaded: ./data/welfare.ttl


In [29]:
# 샘플 쿼리
query = """
    PREFIX schema: <http://schema.org/>

    SELECT ?지역명 ?충전소명 ?위도 ?경도
    WHERE {
        ?atm a schema:AutomatedTeller ;
            schema:name ?충전소명 ;
            schema:addressLocality ?지역 ;
            schema:latitude ?위도 ;
            schema:longitude ?경도 .
        ?지역 schema:name ?지역명 .
        FILTER(CONTAINS(LCASE(?지역명), "서울") && CONTAINS(LCASE(?지역명), "동작구"))
    }
    ORDER BY ?충전소명
"""

# 쿼리 실행
df = run_sparql_query(g, query)
df

,지역명,충전소명,위도,경도
0,서울특별시 동작구,관악동작지사,37.47830829,126.9809567
1,서울특별시 동작구,대방성원 아파트,37.509731742,126.9259166877
2,서울특별시 동작구,대방중간집하장 공영주차장,37.5138842628831,126.930927452442
3,서울특별시 동작구,대우유로카운티,37.5054012556,126.9493141059
4,서울특별시 동작구,동작상떼빌 아파트,37.487899553,126.906937744
5,서울특별시 동작구,래미안상도3차아파트,37.49815,126.953982
6,서울특별시 동작구,보라매아카데미타워 아파트,37.4912743812,126.9240940365
7,서울특별시 동작구,빙수골 장미주차장,37.493303809459974,126.93367320891889
8,서울특별시 동작구,사당롯데캐슬샤인,37.4909601775,126.9715998344
9,서울특별시 동작구,상도대림 아파트,37.4931852401,126.9532620463


In [40]:
# 샘플 쿼리
query = """
    PREFIX schema: <http://schema.org/>

    SELECT * 
    WHERE {
    ?s ?p ?o
    }
"""

# 쿼리 실행
df = run_sparql_query(g, query)
df

,p,o,s
0,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,http://schema.org/BusStop,http://labs.datahub.kr/egs/busStation/GGB20800...
1,http://schema.org/identifier,36460,http://labs.datahub.kr/egs/busStation/TSB34300...
2,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,http://schema.org/BusStop,http://labs.datahub.kr/egs/busStation/ASB28800...
3,http://schema.org/identifier,32010,http://labs.datahub.kr/egs/busStation/CCB25000...
4,http://schema.org/longitude,126.66527295,http://labs.datahub.kr/egs/busStation/GUB2610760
...,...,...,...
1698457,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,http://schema.org/BusStop,http://labs.datahub.kr/egs/busStation/TSB28100...
1698458,http://schema.org/name,은못이사거리,http://labs.datahub.kr/egs/busStation/GGB21800...
1698459,http://schema.org/addressLocality,경기도 안성시,http://labs.datahub.kr/egs/busStation/GGB23100...
1698460,http://purl.org/dc/terms/identifier,ICB166000454,http://labs.datahub.kr/egs/busStation/ICB16600...


In [31]:
# 샘플 쿼리
query = """
PREFIX schema: <http://schema.org/>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

SELECT ?지역명 
       ?면적 
       ?복지수급자수 
       ?버스정류장수 
       ?복지밀도 
       ?버스밀도 
       ?종합점수
WHERE {
  # 1단계: 행정구역과 면적, 인구수
  ?지역 a schema:AdministrativeArea ;
        schema:name ?지역명 ;
        schema:geoWithin ?면적 ;
        schema:population ?인구수 .
  
  BIND(xsd:decimal(?면적) AS ?면적숫자)
  BIND(xsd:decimal(?인구수) AS ?인구숫자)

  # 2단계: 지역별 복지수급자 수 합산
  {
    SELECT ?지역명 (SUM(xsd:decimal(?수급자수)) AS ?복지수급자수)
    WHERE {
      ?복지 a schema:GovernmentService ;
             schema:serviceArea ?지역 ;
             schema:numberOfEmployees ?수급자수 .
      ?지역 schema:name ?지역명 .
    } GROUP BY ?지역명
  }

  # 3단계: 지역별 버스정류장 수 계산
  {
    SELECT ?지역명 (COUNT(?버스) AS ?버스정류장수)
    WHERE {
      ?버스 a schema:BusStop ;
            schema:addressLocality ?지역명 .
    } GROUP BY ?지역명
  }

  # 4단계: 면적 대비 밀도 계산
  BIND((?복지수급자수 / ?면적숫자) AS ?복지밀도)
  BIND((?버스정류장수 / ?면적숫자) AS ?버스밀도)

  # 5단계: 종합 점수 계산 (높은 복지밀도, 낮은 버스밀도 = 취약)
  BIND((?복지밀도 * 10) - (?버스밀도 * 5) AS ?종합점수)
}
ORDER BY DESC(?종합점수)
"""

# 쿼리 실행
df = run_sparql_query(g, query)
df

,지역명,면적,복지수급자수,버스정류장수,복지밀도,버스밀도,종합점수
0,전라남도 목포시,51.73,106389,1154,2056.620916296153102648366518,22.30813841098008892325536439,20454.66847090663058186738836
1,전라남도 목포시,51.73,106389,1154,2056.620916296153102648366518,22.30813841098008892325536439,20454.66847090663058186738836
2,경기도 의정부시,81.55,154266,737,1891.673819742489270386266094,9.037400367872470876762722256,18871.55119558553034947884733
3,경기도 군포시,36.42,65623,462,1801.839648544755628775398133,12.68533772652388797364085667,17954.96979681493684788577705
4,경기도 광명시,38.52,62273,455,1616.640706126687435098650052,11.81204569055036344755970924,16107.34683281412253374870197
...,...,...,...,...,...,...,...
138,경상북도 영양군,815.87,9994,588,12.24950053317317709929277949,0.7207030531824923088237096596,118.8914900658193094488092466
139,강원특별자치도 평창군,1464.26,15177,63,10.36496250665865351781787387,0.04302514580743856965293049049,103.4344993375493423299140862
140,전라남도 목포시,12362.33,106389,1154,8.605901961846998098254940614,0.09334809861894966401964678180,85.59227912537523266245117223
141,전라남도 목포시,12362.33,106389,1154,8.605901961846998098254940614,0.09334809861894966401964678180,85.59227912537523266245117223


In [36]:
def process_welfare_vulnerability_analysis(df):
    
    print(f"입력 데이터: {len(df)}개 행")
    print(f"원본 컬럼: {list(df.columns)}")
    
    # 1단계: 데이터 검증 및 정리
    processed_data = []
    
    for i, row in df.iterrows():
        try:
            # 필수 필드 확인
            region_name = str(row.get('지역명', '')).strip()
            if not region_name or region_name == '':
                print(f"행 {i}: 지역명이 없음, 건너뜀")
                continue
            
            # 숫자 필드 안전 변환
            area = float(row.get('면적', 0) or 0)
            if area <= 0:
                area = 1.0  # 0 나누기 방지
            
            welfare_count = int(float(row.get('복지수급자수', 0) or 0))
            bus_count = int(float(row.get('버스정류장수', 0) or 0))
            
            # 기존 계산값이 있는지 확인, 없으면 직접 계산
            welfare_density = float(row.get('복지밀도', 0) or 0)
            if welfare_density == 0:
                welfare_density = welfare_count / area
            
            bus_density = float(row.get('버스밀도', 0) or 0)
            if bus_density == 0:
                bus_density = bus_count / area
            
            composite_score = float(row.get('종합점수', 0) or 0)
            if composite_score == 0:
                composite_score = (welfare_density * 10) - (bus_density * 5)
            
            # 처리된 행 생성
            processed_row = {
                '지역명': region_name,
                '면적': round(area, 2),
                '복지수급자수': welfare_count,
                '버스정류장수': bus_count,
                '복지밀도': round(welfare_density, 3),
                '버스밀도': round(bus_density, 3),
                '종합점수': round(composite_score, 2)
            }
            
            processed_data.append(processed_row)
            
        except Exception as e:
            print(f"행 {i} 처리 오류: {str(e)}, 건너뜀")
            continue
    
    if not processed_data:
        print("처리 가능한 데이터가 없음")
        return pd.DataFrame()
    
    # DataFrame으로 변환
    processed_df = pd.DataFrame(processed_data)
    print(f"처리 후: {len(processed_df)}개 행")
    
    # 2단계: 중복 제거 (지역명 기준으로 가장 완전한 데이터 선택)
    def select_best_row(group):
        """중복된 지역 중 가장 완전한 데이터 선택"""
        if len(group) == 1:
            return group.iloc[0]
        
        # 완전성 점수 계산 (데이터가 있는 필드 개수)
        completeness_scores = []
        for _, row in group.iterrows():
            score = sum([
                1 if row['복지수급자수'] > 0 else 0,
                1 if row['버스정류장수'] > 0 else 0,
                1 if row['면적'] > 1 else 0
            ])
            completeness_scores.append(score)
        
        # 가장 높은 완전성 점수를 가진 행 선택
        best_idx = completeness_scores.index(max(completeness_scores))
        return group.iloc[best_idx]
    
    # 지역명으로 그룹화하여 중복 제거
    unique_df = processed_df.groupby('지역명').apply(select_best_row).reset_index(drop=True)
    print(f"중복 제거 후: {len(unique_df)}개 지역")
    
    # 3단계: 종합점수 정규화 (0-100 스케일)
    if len(unique_df) > 1:
        scores = unique_df['종합점수'].values
        min_score = scores.min()
        max_score = scores.max()
        score_range = max_score - min_score
        
        if score_range > 0:
            # Min-Max 정규화: (x - min) / (max - min) * 100
            normalized_scores = ((scores - min_score) / score_range) * 100
            unique_df['정규화점수'] = np.round(normalized_scores, 1)
        else:
            # 모든 점수가 동일한 경우
            unique_df['정규화점수'] = 50.0
    else:
        # 결과가 1개인 경우
        unique_df['정규화점수'] = 50.0
    
    # 4단계: 종합점수로 정렬 (취약성 높은 순)
    unique_df = unique_df.sort_values('종합점수', ascending=False).reset_index(drop=True)
    
    # 5단계: 통계 정보 추가
    total_beneficiaries = unique_df['복지수급자수'].sum()
    total_bus_stops = unique_df['버스정류장수'].sum()
    avg_score = unique_df['종합점수'].mean()
    
    print(f"\n=== 분석 결과 ===")
    print(f"최종 지역 수: {len(unique_df)}개")
    print(f"총 복지수급자: {total_beneficiaries:,}명")
    print(f"총 버스정류장: {total_bus_stops:,}개")
    print(f"평균 종합점수: {avg_score:.2f}")
    print(f"가장 취약한 지역: {unique_df.iloc[0]['지역명']}")
    print(f"가장 안전한 지역: {unique_df.iloc[-1]['지역명']}")
    
    return unique_df

# 사용 예시
def example_usage():
    """사용 예시"""
    
    # 가상의 SPARQL 결과 DataFrame 생성 (실제로는 SPARQL 쿼리 결과를 사용)
    sample_data = {
        '지역명': ['강남구', '강북구', '강서구', '강남구', '관악구'],  # 중복 포함
        '면적': [39.5, 23.6, 41.4, 39.5, 29.6],
        '복지수급자수': [1500, 3200, 2800, 1500, 4100],
        '버스정류장수': [85, 45, 62, 85, 38],
        '복지밀도': [0, 0, 0, 0, 0],  # SPARQL에서 계산 안된 경우
        '버스밀도': [0, 0, 0, 0, 0],   # SPARQL에서 계산 안된 경우
        '종합점수': [0, 0, 0, 0, 0]     # SPARQL에서 계산 안된 경우
    }
    
    df = pd.DataFrame(sample_data)
    print("원본 데이터:")
    print(df)
    print("\n" + "="*50 + "\n")
    
    # 처리 실행
    result_df = process_welfare_vulnerability_analysis(df)
    
    print("\n처리된 결과:")
    print(result_df)
    
    return result_df


processed_df = process_welfare_vulnerability_analysis(df)

processed_df

입력 데이터: 143개 행
원본 컬럼: ['지역명', '면적', '복지수급자수', '버스정류장수', '복지밀도', '버스밀도', '종합점수']
처리 후: 143개 행
중복 제거 후: 131개 지역

=== 분석 결과 ===
최종 지역 수: 131개
총 복지수급자: 6,228,938명
총 버스정류장: 170,215개
평균 종합점수: 1815.50
가장 취약한 지역: 전라남도 목포시
가장 안전한 지역: 강원특별자치도 인제군


,지역명,면적,복지수급자수,버스정류장수,복지밀도,버스밀도,종합점수,정규화점수
0,전라남도 목포시,51.73,106389,1154,2056.621,22.308,20454.67,100.0
1,경기도 의정부시,81.55,154266,737,1891.674,9.037,18871.55,92.2
2,경기도 군포시,36.42,65623,462,1801.840,12.685,17954.97,87.7
3,경기도 광명시,38.52,62273,455,1616.641,11.812,16107.35,78.7
4,서울특별시,23.91,37225,4002,1556.880,167.378,14731.91,71.9
...,...,...,...,...,...,...,...,...
126,강원특별자치도 양구군,661.99,8234,2,12.438,0.003,124.37,0.3
127,강원특별자치도 정선군,1219.78,15150,58,12.420,0.048,123.96,0.3
128,경상북도 영양군,815.87,9994,588,12.250,0.721,118.89,0.3
129,강원특별자치도 평창군,1464.26,15177,63,10.365,0.043,103.43,0.2


In [39]:
processed_df.head(10)

,지역명,면적,복지수급자수,버스정류장수,복지밀도,버스밀도,종합점수,정규화점수
0,전라남도 목포시,51.73,106389,1154,2056.621,22.308,20454.67,100.0
1,경기도 의정부시,81.55,154266,737,1891.674,9.037,18871.55,92.2
2,경기도 군포시,36.42,65623,462,1801.840,12.685,17954.97,87.7
3,경기도 광명시,38.52,62273,455,1616.641,11.812,16107.35,78.7
4,서울특별시,23.91,37225,4002,1556.880,167.378,14731.91,71.9
5,경기도 구리시,33.34,48775,335,1462.957,10.048,14579.33,71.2
6,경기도 오산시,42.69,47132,464,1104.052,10.869,10986.18,53.6
7,경기도 시흥시,139.94,113309,1215,809.697,8.682,8053.56,39.2
8,경기도 하남시,92.99,56315,607,605.603,6.528,6023.39,29.2
9,경기도 의왕시,54.03,27794,421,514.418,7.792,5105.22,24.7
